In [1]:
import pandas as pd
pd.set_option("display.max_columns",None)
from datetime import datetime

In [2]:
df_trip = pd.read_csv("./green_tripdata_2019-09.csv",low_memory=False)
df_zone_lookup = pd.read_csv("./taxi+_zone_lookup.csv")

In [3]:
df_trip.lpep_pickup_datetime = pd.to_datetime(df_trip.lpep_pickup_datetime) 
df_trip.lpep_dropoff_datetime = pd.to_datetime(df_trip.lpep_dropoff_datetime) 

In [4]:
len(df_trip[(df_trip.lpep_pickup_datetime>=pd.to_datetime("2019-09-18")) & (df_trip.lpep_dropoff_datetime < pd.to_datetime("2019-09-19") )])

15612

In [5]:
df_trip['pickup_date'] = df_trip.lpep_pickup_datetime.apply(lambda x: x.strftime("%Y-%m-%d") )

In [6]:
df_trip[df_trip['trip_distance']==max(df_trip.trip_distance)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,pickup_date
337292,2.0,2019-09-26 19:32:52,2019-09-27 01:50:27,N,4.0,265,16,1.0,341.64,1761.0,1.0,0.5,0.0,0.0,NaN,0.3,1762.8,2.0,1.0,0.0,2019-09-26


In [7]:
df_zone_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [8]:
df_zone_lookup_pu = df_zone_lookup.copy()
df_zone_lookup_pu.columns = ['PULocationID','PUBorough','PUZone','pu_service_zone']

In [9]:
df_zone_lookup_do = df_zone_lookup.copy()
df_zone_lookup_do.columns = ['DOLocationID','DOBorough','DOZone','do_service_zone']

In [10]:
df_trip_pu = df_trip[df_trip['pickup_date']=='2019-09-18'].merge(df_zone_lookup_pu,on=['PULocationID'],how='inner')

In [11]:
df_trip_pu.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,pickup_date,PUBorough,PUZone,pu_service_zone
0,2.0,2019-09-18 09:12:39,2019-09-18 09:20:14,N,1.0,95,134,1.0,1.38,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2.0,1.0,0.0,2019-09-18,Queens,Forest Hills,Boro Zone
1,2.0,2019-09-18 10:05:23,2019-09-18 10:12:42,N,1.0,95,196,1.0,1.23,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2.0,1.0,0.0,2019-09-18,Queens,Forest Hills,Boro Zone
2,2.0,2019-09-18 10:53:43,2019-09-18 11:00:01,N,1.0,95,196,1.0,1.31,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2.0,1.0,0.0,2019-09-18,Queens,Forest Hills,Boro Zone
3,2.0,2019-09-18 00:01:25,2019-09-18 00:05:35,N,1.0,95,95,1.0,0.79,5.0,0.5,0.5,1.26,0.0,NaN,0.3,7.56,1.0,1.0,0.0,2019-09-18,Queens,Forest Hills,Boro Zone
4,2.0,2019-09-18 11:58:35,2019-09-18 12:10:22,N,1.0,95,102,1.0,3.47,12.5,0.5,0.5,0.00,0.0,NaN,0.3,13.80,2.0,1.0,0.0,2019-09-18,Queens,Forest Hills,Boro Zone


In [12]:
df_pu_fare_amount = df_trip_pu.groupby(['PUBorough'])['fare_amount'].sum().reset_index().sort_values("fare_amount",ascending=False)

In [13]:
df_pu_fare_amount[df_pu_fare_amount["fare_amount"]>50000]

,PUBorough,fare_amount
1,Brooklyn,81795.96
2,Manhattan,72356.18
3,Queens,67243.45


In [14]:
df_sep = df_trip[(df_trip.lpep_pickup_datetime>=pd.to_datetime("2019-09-01")) &
                  (df_trip.lpep_dropoff_datetime < pd.to_datetime("2019-10-01") )].copy()

In [15]:
df_sep = df_sep.merge(df_zone_lookup_pu,on=['PULocationID'],how='inner')
df_sep = df_sep.merge(df_zone_lookup_do,on=['DOLocationID'],how='inner')

In [16]:
df_sep.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,pickup_date,PUBorough,PUZone,pu_service_zone,DOBorough,DOZone,do_service_zone
0,2.0,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1.0,65,189,5.0,2.00,10.5,0.5,0.5,2.36,0.0,NaN,0.3,14.16,1.0,1.0,0.0,2019-09-01,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Prospect Heights,Boro Zone
1,2.0,2019-09-01 12:31:28,2019-09-01 12:38:46,N,1.0,65,189,1.0,1.18,7.0,0.0,0.5,1.56,0.0,NaN,0.3,9.36,1.0,1.0,0.0,2019-09-01,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Prospect Heights,Boro Zone
2,2.0,2019-09-01 14:25:01,2019-09-02 13:56:24,N,1.0,65,189,1.0,1.50,8.0,0.0,0.5,1.32,0.0,NaN,0.3,10.12,1.0,1.0,0.0,2019-09-01,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Prospect Heights,Boro Zone
3,2.0,2019-09-01 17:19:12,2019-09-01 17:24:15,N,1.0,65,189,1.0,0.90,5.5,0.0,0.5,1.26,0.0,NaN,0.3,7.56,1.0,1.0,0.0,2019-09-01,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Prospect Heights,Boro Zone
4,2.0,2019-09-01 18:56:31,2019-09-01 19:05:13,N,1.0,65,189,1.0,1.62,8.5,0.0,0.5,1.86,0.0,NaN,0.3,11.16,1.0,1.0,0.0,2019-09-01,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Prospect Heights,Boro Zone


In [17]:
df_sep = df_sep[df_sep['PUZone']=='Astoria'].copy()

In [18]:
# df_sep = df_sep[df_sep['DOBorough']!='Unknown']

In [22]:
max(df_sep['tip_amount'])

62.31

In [23]:
df_sep[df_sep['tip_amount']==max(df_sep['tip_amount'])]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,pickup_date,PUBorough,PUZone,pu_service_zone,DOBorough,DOZone,do_service_zone
90922,2.0,2019-09-08 18:10:40,2019-09-08 18:44:42,N,1.0,7,132,1.0,14.05,40.0,0.0,0.5,62.31,0.0,NaN,0.3,103.11,1.0,1.0,0.0,2019-09-08,Queens,Astoria,Boro Zone,Queens,JFK Airport,Airports
